## Junta tablas de gopro

In [1]:
import os
import pandas as pd
from datetime import datetime
from utils.get_scan_number import *

In [2]:
# SELECCIONAR QUÉ PRODUCTO QUEREMOS AGRUPAR # solo cambia este chunk
producto = "gopro"
fecha = datetime.now().strftime("%Y%m%d")
#fecha = '20250117'  # Fecha en formato yyyymmdd
carpeta_origen = '../data/1_datos_raw'
carpeta_destino = "../data/2_datos_por_escaneo"

In [ ]:
def read_and_join_csvs_by_product_and_date(fecha, producto, n, folder_path='../data/1_datos_raw'):
    """
    Función que toma una fecha en formato 'yyyymmdd', el nombre del producto, n 
    (el número de escaneo) y hace un join. 
    Añade una columna con la fecha.
    
    Args:
    - fecha (str): Fecha en formato 'yyyymmdd'.
    - producto (str): Nombre del producto a buscar.
    - n (int): Número de escaneo.
    - folder_path (str): Directorio donde se encuentran los archivos CSV. Por defecto, es el directorio '../1_datos_raw'.

    Returns:
    - pd.DataFrame: df
    """
    n = str(n)
    # Listar todos los archivos en la carpeta
    files = os.listdir(folder_path)
    # Filtrar los archivos que coinciden con la fecha, producto y número de escaneo
    csv_files = []
    for f in files:
        # Convertir el nombre del archivo a minúsculas para evitar problemas de mayúsculas/minúsculas
        lower_f = f.lower()
        if lower_f.startswith(fecha) and lower_f.endswith('.csv'):
            parts = lower_f.split('_')
            if len(parts) > 3:
                file_n = parts[1]
                file_product = parts[2] + " " + parts[3]  # Asegura que el producto pueda tener espacios
                # Comprobar que el producto y número de escaneo coincidan
                if producto.lower() in file_product and n == file_n:
                    csv_files.append(f)
    # Verificar que haya más de un archivo para realizar el join
    if len(csv_files) > 1:
        # Leer los archivos con la misma fecha, producto y número de escaneo y añadir la columna "fecha"
        dfs = []
        for f in csv_files:
            df = pd.read_csv(os.path.join(folder_path, f))
            df['fecha'] = fecha  # Añadir la columna con la fecha
            dfs.append(df)
        merged_df = dfs[0]
        for df in dfs[1:]:
            merged_df = merged_df.merge(df, how='outer')  # Outer join para añadir registros no coincidentes
        del merged_df['Unnamed: 0']
        return merged_df
    elif len(csv_files) == 1:
        df = pd.read_csv(os.path.join(folder_path, csv_files[0]))
        df['fecha'] = fecha
        del df['Unnamed: 0']
        return df
    else: 
        return None
# bucle para recorrer todos los escaneos
n = 0
for n in range(0, 9):
    df = read_and_join_csvs_by_product_and_date(fecha, producto, n) # join
    if df is not None:
        df = df.reset_index(drop=True) # sin indice
        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)
        nombre_archivo_csv = carpeta_destino + '/' + producto + '_' + fecha + '_' \
            + str(n) + '.csv'
        df.to_csv(nombre_archivo_csv)
        print(nombre_archivo_csv)

../data/2_datos_por_escaneo/gopro_20250224_1.csv
../data/2_datos_por_escaneo/gopro_20250224_2.csv
